# Import

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


# Fungsi bantu untuk mengambil detail tiap buku

In [ ]:
def get_book_details(book_url):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # --- Ambil kategori ---
    category = soup.select_one("ul.breadcrumb li:nth-of-type(3) a").text.strip()

    # --- Ambil judul ---
    title = soup.find("div", class_="product_main").h1.text.strip()

    # --- Ambil rating ---
    rating = soup.find("p", class_="star-rating")["class"][1]

    # --- Ambil cover image ---
    cover = "https://books.toscrape.com/" + soup.find("div", class_="item active").img["src"].replace("../", "")

    # --- Ambil tabel detail (UPC, harga, stok, review) ---
    table = soup.find("table", class_="table table-striped")
    details = {row.th.text: row.td.text for row in table.find_all("tr")}

    code = details.get("UPC", "")
    price_excl = details.get("Price (excl. tax)", "")
    price_incl = details.get("Price (incl. tax)", "")
    tax = details.get("Tax", "")
    stock_status = "In stock" if "In stock" in details.get("Availability", "") else "Out of stock"

    # Ekstrak jumlah stok tersedia (angka)
    import re
    stock_available = re.findall(r'\d+', details.get("Availability", "0"))
    stock_available = int(stock_available[0]) if stock_available else 0

    # --- Ambil deskripsi (bisa kosong) ---
    desc = soup.select_one("#product_description ~ p")
    description = desc.text.strip() if desc else "No description available"

    # --- Jumlah ulasan ---
    num_reviews = details.get("Number of reviews", "0")

    # Kembalikan sebagai dictionary
    return {
        "category": category,
        "code": code,
        "cover": cover,
        "title": title,
        "rating": rating,
        "price (excl. tax)": price_excl,
        "price (incl. tax)": price_incl,
        "tax": tax,
        "stock status": stock_status,
        "number of stock available": stock_available,
        "description": description,
        "number of reviews": num_reviews,
        "url": book_url
    }

# Loop scraping URL

In [ ]:
base_url = "https://books.toscrape.com/catalogue/page-{}.html"
all_books = []

for page in range(1, 51):  # total 50 halaman (1000 buku)
    print(f"📄 Scraping halaman {page} ...")
    response = requests.get(base_url.format(page))
    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.select("article.product_pod h3 a")

    for b in books:
        book_url = "https://books.toscrape.com/catalogue/" + b["href"]
        try:
            book_data = get_book_details(book_url)
            all_books.append(book_data)
            time.sleep(0.2)  # jeda kecil untuk mencegah overload
        except Exception as e:
            print(f"Gagal ambil data: {book_url} | Error: {e}")
            continue

print("✅ Scraping selesai. Total buku:", len(all_books))

📄 Scraping halaman 1 ...
📄 Scraping halaman 2 ...
📄 Scraping halaman 3 ...
📄 Scraping halaman 4 ...
📄 Scraping halaman 5 ...
📄 Scraping halaman 6 ...
📄 Scraping halaman 7 ...
📄 Scraping halaman 8 ...
📄 Scraping halaman 9 ...
📄 Scraping halaman 10 ...
📄 Scraping halaman 11 ...
📄 Scraping halaman 12 ...
📄 Scraping halaman 13 ...
📄 Scraping halaman 14 ...
📄 Scraping halaman 15 ...
📄 Scraping halaman 16 ...
📄 Scraping halaman 17 ...
📄 Scraping halaman 18 ...
📄 Scraping halaman 19 ...
📄 Scraping halaman 20 ...
📄 Scraping halaman 21 ...
📄 Scraping halaman 22 ...
📄 Scraping halaman 23 ...
📄 Scraping halaman 24 ...
📄 Scraping halaman 25 ...
📄 Scraping halaman 26 ...
📄 Scraping halaman 27 ...
📄 Scraping halaman 28 ...
📄 Scraping halaman 29 ...
📄 Scraping halaman 30 ...
📄 Scraping halaman 31 ...
📄 Scraping halaman 32 ...
📄 Scraping halaman 33 ...
📄 Scraping halaman 34 ...
📄 Scraping halaman 35 ...
📄 Scraping halaman 36 ...
📄 Scraping halaman 37 ...
📄 Scraping halaman 38 ...
📄 Scraping halaman 39

# Membuat DataFrame

In [ ]:
df_books = pd.DataFrame(all_books)

# Hasil DataFrame

In [ ]:
print("Jumlah buku:", len(df_books))
df_books.head()

Jumlah buku: 1000


,category,code,cover,title,rating,price (excl. tax),price (incl. tax),tax,stock status,number of stock available,description,number of reviews,url
0,Poetry,a897fe39b1053632,https://books.toscrape.com/media/cache/fe/72/f...,A Light in the Attic,Three,Â£51.77,Â£51.77,Â£0.00,In stock,22,It's hard to imagine a world without A Light i...,0,https://books.toscrape.com/catalogue/a-light-i...
1,Historical Fiction,90fa61229261140a,https://books.toscrape.com/media/cache/08/e9/0...,Tipping the Velvet,One,Â£53.74,Â£53.74,Â£0.00,In stock,20,"""Erotic and absorbing...Written with starling ...",0,https://books.toscrape.com/catalogue/tipping-t...
2,Fiction,6957f44c3847a760,https://books.toscrape.com/media/cache/ee/cf/e...,Soumission,One,Â£50.10,Â£50.10,Â£0.00,In stock,20,"Dans une France assez proche de la nÃ´tre, un ...",0,https://books.toscrape.com/catalogue/soumissio...
3,Mystery,e00eb4fd7b871a48,https://books.toscrape.com/media/cache/c0/59/c...,Sharp Objects,Four,Â£47.82,Â£47.82,Â£0.00,In stock,20,"WICKED above her hipbone, GIRL across her hear...",0,https://books.toscrape.com/catalogue/sharp-obj...
4,History,4165285e1663650f,https://books.toscrape.com/media/cache/ce/5f/c...,Sapiens: A Brief History of Humankind,Five,Â£54.23,Â£54.23,Â£0.00,In stock,20,From a renowned historian comes a groundbreaki...,0,https://books.toscrape.com/catalogue/sapiens-a...


# Membersihkan karakter "Â" pada mata uang

In [ ]:
# Hapus karakter non-ASCII (seperti Â)
df_books['price (excl. tax)'] = df_books['price (excl. tax)'].str.replace('Â', '', regex=False)
df_books['price (incl. tax)'] = df_books['price (incl. tax)'].str.replace('Â', '', regex=False)
df_books['tax'] = df_books['tax'].str.replace('Â', '', regex=False)

# Opsional: ubah menjadi float tanpa simbol £
for col in ['price (excl. tax)', 'price (incl. tax)', 'tax']:
    df_books[col] = df_books[col].str.replace('£', '', regex=False).astype(float)

# Hasil DataFrame setelah pembersihan

In [ ]:
df_books.head()

,category,code,cover,title,rating,price (excl. tax),price (incl. tax),tax,stock status,number of stock available,description,number of reviews,url
0,Poetry,a897fe39b1053632,https://books.toscrape.com/media/cache/fe/72/f...,A Light in the Attic,Three,51.77,51.77,0.0,In stock,22,It's hard to imagine a world without A Light i...,0,https://books.toscrape.com/catalogue/a-light-i...
1,Historical Fiction,90fa61229261140a,https://books.toscrape.com/media/cache/08/e9/0...,Tipping the Velvet,One,53.74,53.74,0.0,In stock,20,"""Erotic and absorbing...Written with starling ...",0,https://books.toscrape.com/catalogue/tipping-t...
2,Fiction,6957f44c3847a760,https://books.toscrape.com/media/cache/ee/cf/e...,Soumission,One,50.10,50.10,0.0,In stock,20,"Dans une France assez proche de la nÃ´tre, un ...",0,https://books.toscrape.com/catalogue/soumissio...
3,Mystery,e00eb4fd7b871a48,https://books.toscrape.com/media/cache/c0/59/c...,Sharp Objects,Four,47.82,47.82,0.0,In stock,20,"WICKED above her hipbone, GIRL across her hear...",0,https://books.toscrape.com/catalogue/sharp-obj...
4,History,4165285e1663650f,https://books.toscrape.com/media/cache/ce/5f/c...,Sapiens: A Brief History of Humankind,Five,54.23,54.23,0.0,In stock,20,From a renowned historian comes a groundbreaki...,0,https://books.toscrape.com/catalogue/sapiens-a...


In [ ]:
df_books.to_csv("books_toscrape_complete.csv", index=False)
print("✅ Dataset lengkap tersimpan sebagai 'books_toscrape_complete.csv'")

✅ Dataset lengkap tersimpan sebagai 'books_toscrape_complete.csv'
